# License Register ACT  
https://www.accesscanberra.act.gov.au/s/public-registers

# Fetch `Disciplinary Register` & `Work assessors`

https://www.accesscanberra.act.gov.au/s/contstruction-professionals

In [1]:
import selenium
import time
from selenium import webdriver

from bs4 import BeautifulSoup
import pandas as pd

## Ignoring Warnings

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
driver = webdriver.Chrome('/Users/rohit_manral/Downloads/chromedriver')

# Getting the access of `ACT Builder's` official website

In [4]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=m0I4a00000000CHEAY')
time.sleep(2)

#### Finding number of pages of the data 

In [6]:
pages=driver.find_element_by_xpath('/html/body/div[3]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages

numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1]) #no. of pages

460


## First Page

### Main program to get `Headers` & `all the data rows` of just the `first page`

In [7]:
page_source = driver.page_source
soup =  BeautifulSoup(page_source, 'lxml')
            
headers = [i.text for i in soup.findAll('th')]
headers = headers[:-1]
print(headers)

['Name', 'Licence No', 'Expiry Date', 'Occupation', 'Class Description']


In [8]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[3]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[6]/lightning-button/button')
#print(xpaths) 

## All pages now

### Main program to get `all the data rows` of all the pages `till last page`

In [9]:
licensee_detail=[]

In [10]:
for i in range(1, numbers[1]+1):
    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    
    data = [j.text for j in soup.findAll('td')]
    all_names= data[::6]
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(0,all_names[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[3]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
        
    # click to next page
    driver.find_element_by_xpath('/html/body/div[3]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr[4]/td[6]/lightning-button/button"}
  (Session info: chrome=107.0.5304.87)
Stacktrace:
0   chromedriver                        0x00000001029802c8 chromedriver + 4752072
1   chromedriver                        0x0000000102900463 chromedriver + 4228195
2   chromedriver                        0x0000000102563b18 chromedriver + 441112
3   chromedriver                        0x00000001025a0e21 chromedriver + 691745
4   chromedriver                        0x00000001025a1061 chromedriver + 692321
5   chromedriver                        0x00000001025dc5e4 chromedriver + 935396
6   chromedriver                        0x00000001025c1d2d chromedriver + 826669
7   chromedriver                        0x00000001025da134 chromedriver + 926004
8   chromedriver 

In [11]:
licensee_detail[0]

['2NV CONSTRUCTIONS PTY LTD',
 '2019305',
 '19 May 2023',
 'Builder',
 'Class C',
 'Valid for the construction of Building Code of Australia (BCA) Classes 1a and 10 only.']

In [12]:
licensee_detail[-1]

['ZYCON BUILDING PTY LTD',
 '2016957',
 '6 October 2025',
 'Builder',
 'Class B',
 '']

In [13]:
len(licensee_detail)

4593

In [14]:
df_builder = pd.DataFrame(licensee_detail, columns = ['Name of Individual or Company', 'License Number','Expiry Date','Occupation','Class','Class Condition / Endorsement'])
df_builder

,Name of Individual or Company,License Number,Expiry Date,Occupation,Class,Class Condition / Endorsement
0,2NV CONSTRUCTIONS PTY LTD,2019305,19 May 2023,Builder,Class C,Valid for the construction of Building Code of...
1,35 DEGREES PTY LTD,20151084,6 December 2024,Builder,Class C,
2,8 9 CONSTRUCTION PTY LTD,2022817,26 August 2025,Builder,Class A,Valid for Building Code of Australia (BCA) cla...
3,A & A CONTRACTORS PTY LIMITED,19968065,5 September 2023,Builder,Class C,
4,A & B DAL CORTIVO PTY LTD,2008265,12 May 2024,Builder,Class C,
...,...,...,...,...,...,...
4588,"ZIVKO, IVAN",19884304,14 July 2024,Builder,Class C,
4589,"ZMISA, SASHA DAVID",2019547,22 September 2023,Builder,Class A,Restricted to Building Codes of Australia (BCA...
4590,"ZOUMAS, DIMITRI JAMES",20211487,19 November 2022,Builder,Class C,Valid for Building Code of Australia (BCA) cla...
4591,"ZUBOVIC, ZELJKO THOMAS",19946847,5 December 2024,Builder,Class C,


In [15]:
df_builder.shape

(4593, 6)

In [16]:
df_builder.to_csv('ACT_Builders.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Electricians

In [36]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=m0I4a00000000BzEAI')
#time.sleep(1)

In [38]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

507


### Generating xpaths for all the records in any page

In [39]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[5]/lightning-button/button')
#print(xpaths) 

### Main program to get `Headers` & `all the data rows` of all the pages till last page

In [40]:
licensee_detail=[]

In [41]:
for i in range(1, numbers[1]+1):
    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::5]
    #all_names
    
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(0,all_names[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
        
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr[3]/td[5]/lightning-button/button"}
  (Session info: chrome=96.0.4664.55)


In [42]:
licensee_detail[0]

['360 DEGREE FIRE PTY LTD',
 '2017990',
 '15/11/2023',
 'Electrician',
 'Electrical Contractor',
 '']

In [43]:
licensee_detail[-1]

['ZVARIMWA, REMEDY',
 '20211044',
 '30/7/2022',
 'Electrician',
 'Unrestricted',
 '']

In [44]:
len(licensee_detail)

5062

In [45]:
df_electrician = pd.DataFrame(licensee_detail, columns = ['Name of Individual or Company', 'License Number','Expiry Date','Occupation','Class','Conditions or Endorsements'])

,Name of Individual or Company,License Number,Expiry Date,Occupation,Class,Conditions or Endorsements
0,360 DEGREE FIRE PTY LTD,2017990,15/11/2023,Electrician,Electrical Contractor,
1,5R ELECTRICAL PTY LTD,2020170,6/3/2023,Electrician,Electrical Contractor,
2,A & K ELECTRICAL (AUSTRALIA) PTY LTD,2007284,27/4/2022,Electrician,Electrical Contractor,
3,A GOOD SPARKY PTY LTD,20151055,2/12/2023,Electrician,Electrical Contractor,Also valid for interval metering work.
4,A-LINE ELECTRICAL SOLUTIONS PTY LTD,2014592,20/6/2023,Electrician,Electrical Contractor,


In [224]:
df_electrician

,Name of Individual or Company,License Number,Expiry Date,Occupation,Class,Conditions or Endorsements
0,360 DEGREE FIRE PTY LTD,2017990,15/11/2023,Electrician,Electrical Contractor,
1,5R ELECTRICAL PTY LTD,2020170,6/3/2023,Electrician,Electrical Contractor,
2,A & K ELECTRICAL (AUSTRALIA) PTY LTD,2007284,27/4/2022,Electrician,Electrical Contractor,
3,A GOOD SPARKY PTY LTD,20151055,2/12/2023,Electrician,Electrical Contractor,Also valid for interval metering work.
4,A-LINE ELECTRICAL SOLUTIONS PTY LTD,2014592,20/6/2023,Electrician,Electrical Contractor,
5,A.G. COOMBS (NSW) PTY. LTD.,2020154,2/4/2023,Electrician,Electrical Contractor,
6,"ABBAS, MAHMOUD",20201728,11/1/2022,Electrician,Unrestricted,
7,"ABBAS, BILLAL",2012560,15/12/2021,Electrician,Unrestricted,
8,"ABBASIPOUR, HOSSEIN",20211297,6/9/2024,Electrician,Unrestricted,
9,"ABBOT, BENJAMIN DALTON",2017738,13/9/2023,Electrician,Unrestricted,


In [225]:
df_electrician.shape

(5062, 6)

In [226]:
df_electrician.to_csv('ACT_Electricians.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Plumbers

In [53]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=m0I4a00000000CMEAY')
#time.sleep(1)

In [54]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

220


In [55]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[5]/lightning-button/button')
#print(xpaths) 

In [56]:
licensee_detail=[]

for i in range(1, numbers[1]+1):
    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::5]
    #all_names
    data= data[3:]
    #data
    all_class_desc= []
    all_class_desc= data[::5]
    #all_class_desc
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(0,all_class_desc[k-1])
        ab.insert(0,all_names[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
        
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr[10]/td[5]/lightning-button/button"}
  (Session info: chrome=96.0.4664.55)


In [67]:
licensee_detail[0]

['360 DEGREE FIRE PTY LTD',
 'Water Supply Plumber',
 '2017990',
 'Plumber',
 '22 January 2023',
 '']

In [68]:
licensee_detail[-1]

['ZOUCH, GEORGE ANDREW',
 'Sanitary Plumber',
 '201583',
 'Plumber',
 '1 June 2024',
 '',
 '']

In [75]:
len(licensee_detail)

2199

In [112]:
df_plumber = pd.DataFrame(licensee_detail, columns = ['Name of Individual or Company','Occupation', 'License Number','Class','Expiry Date','Conditions or Endorsements','x1','x2'])
#df_plumber = df_plumber.drop(['x1','x2'], 1) 
df_plumber 

,Name of Individual or Company,Occupation,License Number,Class,Expiry Date,Conditions or Endorsements,x1,x2
0,360 DEGREE FIRE PTY LTD,Water Supply Plumber,2017990,Plumber,22 January 2023,,None,None
1,A & A HOTWATER & GAS SOLUTIONS PTY LTD,Water Supply Plumber,20191321,Plumber,2 February 2024,,None,None
2,A & K (ACT) PTY LTD,Sanitary Plumber,2016855,Plumber,13 September 2022,,,None
3,A & K (ACT) PTY LTD,Water Supply Plumber,2016855,Plumber,13 September 2022,,,None
4,A PLUS PLUMBING (ACT) PTY LTD,Water Supply Plumber,2016683,Plumber,17 August 2022,,,None
5,A PLUS PLUMBING (ACT) PTY LTD,Sanitary Plumber,2016683,Plumber,17 August 2022,,,None
6,AAA ABOVE BOARD PLUMBING PTY LTD,Water Supply Plumber,2018299,Plumber,31 May 2022,,,None
7,AAA ABOVE BOARD PLUMBING PTY LTD,Sanitary Plumber,2018299,Plumber,31 May 2022,,,None
8,"ABBEY, DAMIEN LETTON",Sanitary Plumber,200216119,Plumber,17 February 2024,Certified to test Backflow Prevention Devices.,None,None
9,"ABDEL RAHMAN, MUDATHIR NOURIN ALI",Journeyperson Plumber,20201405,Plumber,31 October 2023,,None,None


In [181]:
df_plumber

,Name of Individual or Company,Occupation,License Number,Class,Expiry Date,Conditions or Endorsements
0,360 DEGREE FIRE PTY LTD,Water Supply Plumber,2017990,Plumber,22 January 2023,
1,A & A HOTWATER & GAS SOLUTIONS PTY LTD,Water Supply Plumber,20191321,Plumber,2 February 2024,
2,A & K (ACT) PTY LTD,Sanitary Plumber,2016855,Plumber,13 September 2022,
3,A & K (ACT) PTY LTD,Water Supply Plumber,2016855,Plumber,13 September 2022,
4,A PLUS PLUMBING (ACT) PTY LTD,Water Supply Plumber,2016683,Plumber,17 August 2022,
5,A PLUS PLUMBING (ACT) PTY LTD,Sanitary Plumber,2016683,Plumber,17 August 2022,
6,AAA ABOVE BOARD PLUMBING PTY LTD,Water Supply Plumber,2018299,Plumber,31 May 2022,
7,AAA ABOVE BOARD PLUMBING PTY LTD,Sanitary Plumber,2018299,Plumber,31 May 2022,
8,"ABBEY, DAMIEN LETTON",Sanitary Plumber,200216119,Plumber,17 February 2024,Certified to test Backflow Prevention Devices.
9,"ABDEL RAHMAN, MUDATHIR NOURIN ALI",Journeyperson Plumber,20201405,Plumber,31 October 2023,


In [113]:
df_plumber['x1'][21]

'Certified to test Backflow Prevention Devices.'

There is a problem with the data, as 2 different endorsements for a particular license number could be the case. So, first, we will blend the endorsements for each row in a single column. Then, Group By the data with license number.

In [114]:
for i in range(0, len(licensee_detail)):
    if df_plumber['x1'][i]:
        df_plumber['Conditions or Endorsements'][i] = df_plumber['x1'][i]
        df_plumber['x1'][i]='--'
        

In [115]:
df_plumber

,Name of Individual or Company,Occupation,License Number,Class,Expiry Date,Conditions or Endorsements,x1,x2
0,360 DEGREE FIRE PTY LTD,Water Supply Plumber,2017990,Plumber,22 January 2023,,None,None
1,A & A HOTWATER & GAS SOLUTIONS PTY LTD,Water Supply Plumber,20191321,Plumber,2 February 2024,,None,None
2,A & K (ACT) PTY LTD,Sanitary Plumber,2016855,Plumber,13 September 2022,,,None
3,A & K (ACT) PTY LTD,Water Supply Plumber,2016855,Plumber,13 September 2022,,,None
4,A PLUS PLUMBING (ACT) PTY LTD,Water Supply Plumber,2016683,Plumber,17 August 2022,,,None
5,A PLUS PLUMBING (ACT) PTY LTD,Sanitary Plumber,2016683,Plumber,17 August 2022,,,None
6,AAA ABOVE BOARD PLUMBING PTY LTD,Water Supply Plumber,2018299,Plumber,31 May 2022,,,None
7,AAA ABOVE BOARD PLUMBING PTY LTD,Sanitary Plumber,2018299,Plumber,31 May 2022,,,None
8,"ABBEY, DAMIEN LETTON",Sanitary Plumber,200216119,Plumber,17 February 2024,Certified to test Backflow Prevention Devices.,None,None
9,"ABDEL RAHMAN, MUDATHIR NOURIN ALI",Journeyperson Plumber,20201405,Plumber,31 October 2023,,None,None


In [116]:
df_plumber=df_plumber.drop(['x1', 'x2'], axis=1)

In [221]:
df_plumber

,Name of Individual or Company,Occupation,License Number,Class,Expiry Date,Conditions or Endorsements
0,360 DEGREE FIRE PTY LTD,Water Supply Plumber,2017990,Plumber,22 January 2023,
1,A & A HOTWATER & GAS SOLUTIONS PTY LTD,Water Supply Plumber,20191321,Plumber,2 February 2024,
2,A & K (ACT) PTY LTD,Sanitary Plumber,2016855,Plumber,13 September 2022,
3,A & K (ACT) PTY LTD,Water Supply Plumber,2016855,Plumber,13 September 2022,
4,A PLUS PLUMBING (ACT) PTY LTD,Water Supply Plumber,2016683,Plumber,17 August 2022,
5,A PLUS PLUMBING (ACT) PTY LTD,Sanitary Plumber,2016683,Plumber,17 August 2022,
6,AAA ABOVE BOARD PLUMBING PTY LTD,Water Supply Plumber,2018299,Plumber,31 May 2022,
7,AAA ABOVE BOARD PLUMBING PTY LTD,Sanitary Plumber,2018299,Plumber,31 May 2022,
8,"ABBEY, DAMIEN LETTON",Sanitary Plumber,200216119,Plumber,17 February 2024,Certified to test Backflow Prevention Devices.
9,"ABDEL RAHMAN, MUDATHIR NOURIN ALI",Journeyperson Plumber,20201405,Plumber,31 October 2023,


In [222]:
df_plumber.shape

(2199, 6)

In [223]:
df_plumber.to_csv('ACT_Plumbers.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Drainers

In [153]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=m0I4a00000000CJEAY')
#time.sleep(1)

In [154]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

133


In [155]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[5]/lightning-button/button')
#print(xpaths) 

In [156]:
licensee_detail=[]

for i in range(1, numbers[1]+1):
    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::5]
    #all_names
    data= data[3:]
    #data
    all_class_desc= []
    all_class_desc= data[::5]
    #all_class_desc
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(0,all_class_desc[k-1])
        ab.insert(0,all_names[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr[2]/td[5]/lightning-button/button"}
  (Session info: chrome=96.0.4664.55)


In [157]:
licensee_detail[0]

['A & K (ACT) PTY LTD',
 'Advanced Sanitary Drainer',
 '2016855',
 'Drainer',
 '13 September 2022',
 '']

In [158]:
licensee_detail[-1]

['ZOUCH, GEORGE ANDREW',
 'Advanced Sanitary Drainer',
 '201583',
 'Drainer',
 '1 June 2024',
 '']

In [159]:
len(licensee_detail)

1321

In [161]:
df_drainer = pd.DataFrame(licensee_detail, columns = ['Name of Individual or Company','Class description', 'License Number','Occupation','Expiry Date','Conditions or Endorsements'])
df_drainer

,Name of Individual or Company,Class description,License Number,Occupation,Expiry Date,Conditions or Endorsements
0,A & K (ACT) PTY LTD,Advanced Sanitary Drainer,2016855,Drainer,13 September 2022,
1,A PLUS PLUMBING (ACT) PTY LTD,Advanced Sanitary Drainer,2016683,Drainer,17 August 2022,
2,AAA ABOVE BOARD PLUMBING PTY LTD,Advanced Sanitary Drainer,2018299,Drainer,31 May 2022,
3,"ABBEY, DAMIEN LETTON",Advanced Sanitary Drainer,200216119,Drainer,17 February 2024,
4,"ABDEL RAHMAN, MUDATHIR NOURIN ALI",Journeyperson Drainer,20201405,Drainer,31 October 2023,
5,"ABSOLOM, ALEX",Journeyperson Drainer,2021915,Drainer,27 July 2024,
6,"ACKLAND, CHRISTOPHER LEE",Advanced Sanitary Drainer,2014830,Drainer,5 April 2023,
7,"ADAMS, JAMES",Journeyperson Drainer,2021668,Drainer,8 August 2024,
8,ADVANCED PLUMBING & DRAINS (CANBERRA) PTY LTD,Advanced Sanitary Drainer,2017904,Drainer,6 November 2023,
9,"AGGARWAL, SURESH",Advanced Sanitary Drainer,20151,Drainer,20 July 2023,


In [183]:
df_drainer

,Name of Individual or Company,Class description,License Number,Occupation,Expiry Date,Conditions or Endorsements
0,A & K (ACT) PTY LTD,Advanced Sanitary Drainer,2016855,Drainer,13 September 2022,
1,A PLUS PLUMBING (ACT) PTY LTD,Advanced Sanitary Drainer,2016683,Drainer,17 August 2022,
2,AAA ABOVE BOARD PLUMBING PTY LTD,Advanced Sanitary Drainer,2018299,Drainer,31 May 2022,
3,"ABBEY, DAMIEN LETTON",Advanced Sanitary Drainer,200216119,Drainer,17 February 2024,
4,"ABDEL RAHMAN, MUDATHIR NOURIN ALI",Journeyperson Drainer,20201405,Drainer,31 October 2023,
5,"ABSOLOM, ALEX",Journeyperson Drainer,2021915,Drainer,27 July 2024,
6,"ACKLAND, CHRISTOPHER LEE",Advanced Sanitary Drainer,2014830,Drainer,5 April 2023,
7,"ADAMS, JAMES",Journeyperson Drainer,2021668,Drainer,8 August 2024,
8,ADVANCED PLUMBING & DRAINS (CANBERRA) PTY LTD,Advanced Sanitary Drainer,2017904,Drainer,6 November 2023,
9,"AGGARWAL, SURESH",Advanced Sanitary Drainer,20151,Drainer,20 July 2023,


In [184]:
df_drainer['Conditions or Endorsements'].unique()

array(['', 'Also valid to Certified Backflow Prevention Devices',
       'Restricted to work performed as an employee only. Not able to contract.',
       'Certified to Test Backflow Prevention Devices',
       'Certified to Test Backflow Prevention Devices.',
       'Certified to test Backflow Prevention Devices.',
       'Joureyperson drainer can only provide this construction service as an employee under supervision',
       'Certified to test and maintain Backflow Prevention Devices.',
       'Valid for work performed as an employee/Nominee only. Not able to contract',
       'Please contact Access Canberra for condition details',
       'Only valid for work as an employee, not able to contract.', 'NIL',
       'Journeyperson can only provide this construction service as an employee under supervision.',
       'Valid for work performed as an employee/Nominee only. Not able to contract. Valid for work performed as an employee/Nominee only. Not able to contract.',
       'Restricted 

In [219]:
df_drainer.shape

(1321, 6)

In [220]:
df_drainer.to_csv('ACT_Drainers.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Gasfitters

In [200]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=m0I4a00000000CLEAY')
#time.sleep(1)

In [201]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

170


In [202]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[5]/lightning-button/button')
#print(xpaths) 

In [203]:
licensee_detail=[]

for i in range(1, numbers[1]+1):
    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::5]
    #all_names
    data= data[3:]
    #data
    all_class_desc= []
    all_class_desc= data[::5]
    #all_class_desc
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(3,all_class_desc[k-1])
        ab.insert(0,all_names[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr[2]/td[5]/lightning-button/button"}
  (Session info: chrome=96.0.4664.55)


In [204]:
licensee_detail[0]

['A & A HOTWATER & GAS SOLUTIONS PTY LTD',
 '20191321',
 'Gasfitter',
 '2 February 2024',
 'LPG Gasfitter (Vapour Phase)',
 '',
 '']

In [205]:
licensee_detail[-2]

['ZOUCH, GEORGE ANDREW',
 '201583',
 'Gasfitter',
 '1 June 2024',
 'General',
 '',
 '']

In [206]:
len(licensee_detail)

1691

In [209]:
df_gasfitter = pd.DataFrame(licensee_detail, columns = ['Name', 'License number','Occupation','Expiry Date','Class description','Conditions or Endorsements','x2'])
#df_gasfitter = df_gasfitter.drop(['x1','x2'], 1)
df_gasfitter

,Name,License number,Occupation,Expiry Date,Class description,Conditions or Endorsements,x2
0,A & A HOTWATER & GAS SOLUTIONS PTY LTD,20191321,Gasfitter,2 February 2024,LPG Gasfitter (Vapour Phase),,
1,A & A HOTWATER & GAS SOLUTIONS PTY LTD,20191321,Gasfitter,2 February 2024,General,,
2,A & K (ACT) PTY LTD,2016855,Gasfitter,13 September 2022,General,,None
3,A PLUS PLUMBING (ACT) PTY LTD,2016683,Gasfitter,17 August 2022,Advanced,,None
4,"ABBEY, DAMIEN LETTON",200216119,Gasfitter,17 February 2024,General,,
5,"ABBEY, DAMIEN LETTON",200216119,Gasfitter,17 February 2024,LPG Gasfitter (Vapour Phase),,
6,"ABDEL RAHMAN, MUDATHIR NOURIN ALI",20201405,Gasfitter,31 October 2023,Journeyperson Gasfitter,,None
7,"ABSOLOM, ALEX",2021915,Gasfitter,27 July 2024,Journeyperson Gasfitter,,None
8,"ACKLAND, CHRISTOPHER LEE",2014830,Gasfitter,5 April 2023,General,,None
9,"ADAMS, JAMES",2021668,Gasfitter,8 August 2024,Journeyperson Gasfitter,,None


In [210]:
for i in range(0, len(licensee_detail)):
    if df_gasfitter['x2'][i]:
        df_gasfitter['Conditions or Endorsements'][i] = df_gasfitter['x2'][i]
        df_gasfitter['x2'][i]='--'
        
df_gasfitter

,Name,License number,Occupation,Expiry Date,Class description,Conditions or Endorsements,x2
0,A & A HOTWATER & GAS SOLUTIONS PTY LTD,20191321,Gasfitter,2 February 2024,LPG Gasfitter (Vapour Phase),,
1,A & A HOTWATER & GAS SOLUTIONS PTY LTD,20191321,Gasfitter,2 February 2024,General,,
2,A & K (ACT) PTY LTD,2016855,Gasfitter,13 September 2022,General,,None
3,A PLUS PLUMBING (ACT) PTY LTD,2016683,Gasfitter,17 August 2022,Advanced,,None
4,"ABBEY, DAMIEN LETTON",200216119,Gasfitter,17 February 2024,General,,
5,"ABBEY, DAMIEN LETTON",200216119,Gasfitter,17 February 2024,LPG Gasfitter (Vapour Phase),,
6,"ABDEL RAHMAN, MUDATHIR NOURIN ALI",20201405,Gasfitter,31 October 2023,Journeyperson Gasfitter,,None
7,"ABSOLOM, ALEX",2021915,Gasfitter,27 July 2024,Journeyperson Gasfitter,,None
8,"ACKLAND, CHRISTOPHER LEE",2014830,Gasfitter,5 April 2023,General,,None
9,"ADAMS, JAMES",2021668,Gasfitter,8 August 2024,Journeyperson Gasfitter,,None


In [215]:
df_gasfitter['Conditions or Endorsements'].unique()

array(['',
       'Restricted to work performed as an employee only. Not able to contract.',
       'Journeyperson gasfitter can only provide this construction service as an employee under supervision',
       'Please contact Access Canberra for condition details',
       'Gasfitting pressure below 21KPA. LPG Gasfitting pressure below 21KPA.',
       'Restricted to work as an employee only. Not able to contract.',
       'Includes LPG (Vapour Phase)', 'Includes LPG Vapour Phase.',
       'includes LPG vapour phase.', 'Includes LPG Vapour phase.',
       'May only provide gasfitting service as an employee',
       'Fire Sprinkler work restricted to hydrants, hosereels and backflow',
       'Only valid for work as an employee, not able to contract.', 'NIL',
       'Includes LPG vapour phase.', 'includes LPG vapour phase',
       'includes LPG Vapour phase',
       'Journeyperson can only provide this construction service as an employee under supervision.',
       'Includes LPG Vapour Pha

In [216]:
df_gasfitter=df_gasfitter.drop(['x2'], axis=1)
df_gasfitter

,Name,License number,Occupation,Expiry Date,Class description,Conditions or Endorsements
0,A & A HOTWATER & GAS SOLUTIONS PTY LTD,20191321,Gasfitter,2 February 2024,LPG Gasfitter (Vapour Phase),
1,A & A HOTWATER & GAS SOLUTIONS PTY LTD,20191321,Gasfitter,2 February 2024,General,
2,A & K (ACT) PTY LTD,2016855,Gasfitter,13 September 2022,General,
3,A PLUS PLUMBING (ACT) PTY LTD,2016683,Gasfitter,17 August 2022,Advanced,
4,"ABBEY, DAMIEN LETTON",200216119,Gasfitter,17 February 2024,General,
5,"ABBEY, DAMIEN LETTON",200216119,Gasfitter,17 February 2024,LPG Gasfitter (Vapour Phase),
6,"ABDEL RAHMAN, MUDATHIR NOURIN ALI",20201405,Gasfitter,31 October 2023,Journeyperson Gasfitter,
7,"ABSOLOM, ALEX",2021915,Gasfitter,27 July 2024,Journeyperson Gasfitter,
8,"ACKLAND, CHRISTOPHER LEE",2014830,Gasfitter,5 April 2023,General,
9,"ADAMS, JAMES",2021668,Gasfitter,8 August 2024,Journeyperson Gasfitter,


In [217]:
df_gasfitter.shape

(1691, 6)

In [218]:
df_gasfitter.to_csv('ACT_Gasfitter.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Asbestos Removal

In [228]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=m0I4a00000000CGEAY')
time.sleep(1)

In [244]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

7


In [245]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[6]/lightning-button/button')
#print(xpaths) 

In [246]:
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::6]
    all_names

['3D DEMOLITION PTY LTD',
 'ACE DEMOLITION & EXCAVATION PTY LTD',
 'AGH DEMOLITION & ASBESTOS REMOVAL PTY LTD',
 'ANZ ENVIRO PTY LTD',
 'ASBESTOS AUSTRALIA PTY LTD',
 'AUSCOMM PTY LTD',
 'AUSTRALASIAN TECHNICAL SERVICES NSW PTY LTD',
 'AZTECH SERVICES AUSTRALIA PTY LTD',
 'BC LIM INVESTMENTS PTY LTD',
 'BEASY PTY LTD']

In [247]:
licensee_detail=[]

for i in range(1, numbers[1]+1):
    

    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::6]
    #all_names
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(0,all_names[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr[9]/td[6]/lightning-button/button"}
  (Session info: chrome=96.0.4664.55)


In [248]:
licensee_detail[0]

['3D DEMOLITION PTY LTD',
 'AD213117',
 'NSW',
 'CLASS A',
 '1/10/2024',
 'HURSTVILLE  GROVE, NSW',
 'Asbestos removalist',
 'All friable asbestos removal work  All non-friable asbestos removal work']

In [249]:
licensee_detail[-1]

['WORLDWIDE DEMOLITIONS PTY LTD',
 'AD211218',
 'NSW',
 'CLASS B',
 '',
 'OAKS FLATS, NSW',
 'Asbestos removalist',
 'Nil']

In [250]:
len(licensee_detail)

68

In [251]:
df_abs_rem = pd.DataFrame(licensee_detail, columns = ['Name', 'License number','State','Class','Expiry Date','Business Location','Occupation','Class Condition / Endorsement'])
df_abs_rem

,Name,License number,State,Class,Expiry Date,Business Location,Occupation,Class Condition / Endorsement
0,3D DEMOLITION PTY LTD,AD213117,NSW,CLASS A,1/10/2024,"HURSTVILLE GROVE, NSW",Asbestos removalist,All friable asbestos removal work All non-fri...
1,ACE DEMOLITION & EXCAVATION PTY LTD,AD211699,NSW,CLASS B,,"REGENTS PARK, NSW",Asbestos removalist,Nil
2,AGH DEMOLITION & ASBESTOS REMOVAL PTY LTD,ARA00011,ACT,CLASS A,2/12/2023,"QUEANBEYAN, NSW",Asbestos removalist,Nil
3,ANZ ENVIRO PTY LTD,91040716,TAS,CLASS A,,"MALAGA, WA",Asbestos removalist,Nil
4,ASBESTOS AUSTRALIA PTY LTD,H17/00884,VIC,CLASS A,,"KEYSBOROUGH, VIC",Asbestos removalist,NIL
5,AUSCOMM PTY LTD,2319370,QLD,CLASS B,,"KULIKUP, WA",Asbestos removalist,The licence holder must ensure that an asbesto...
6,AUSTRALASIAN TECHNICAL SERVICES NSW PTY LTD,AD212177,NSW,CLASS A,6/4/2025,"INGLEBURN, NSW",Asbestos removalist,All friable asbestos removal work All non-fri...
7,AZTECH SERVICES AUSTRALIA PTY LTD,2313682,QLD,CLASS A,,"YATALA, QLD",Asbestos removalist,The licence holder must ensure that an asbesto...
8,BC LIM INVESTMENTS PTY LTD,H18/02873,VIC,CLASS B,,"TULLAMARINE, VIC",Asbestos removalist,Limited to the removal of Non-friable asbestos...
9,BEASY PTY LTD,AD211426,NSW,CLASS A,5/12/2023,"YENNORA, NSW",Asbestos removalist,Nil


In [252]:
df_abs_rem.shape

(68, 8)

#### Managing the Class attribute

In [253]:
import pandas as pd

In [254]:
list(df_abs_rem['Class'].unique())

['CLASS A', 'CLASS B', 'Class B']

In [255]:
all_class=[]

for i in df_abs_rem['Class']:
    all_class.append(i[-1:])
    
df_abs_rem['Class']=all_class
list(df_abs_rem['Class'].unique())

['A', 'B']

In [256]:
df_abs_rem

,Name,License number,State,Class,Expiry Date,Business Location,Occupation,Class Condition / Endorsement
0,3D DEMOLITION PTY LTD,AD213117,NSW,A,1/10/2024,"HURSTVILLE GROVE, NSW",Asbestos removalist,All friable asbestos removal work All non-fri...
1,ACE DEMOLITION & EXCAVATION PTY LTD,AD211699,NSW,B,,"REGENTS PARK, NSW",Asbestos removalist,Nil
2,AGH DEMOLITION & ASBESTOS REMOVAL PTY LTD,ARA00011,ACT,A,2/12/2023,"QUEANBEYAN, NSW",Asbestos removalist,Nil
3,ANZ ENVIRO PTY LTD,91040716,TAS,A,,"MALAGA, WA",Asbestos removalist,Nil
4,ASBESTOS AUSTRALIA PTY LTD,H17/00884,VIC,A,,"KEYSBOROUGH, VIC",Asbestos removalist,NIL
5,AUSCOMM PTY LTD,2319370,QLD,B,,"KULIKUP, WA",Asbestos removalist,The licence holder must ensure that an asbesto...
6,AUSTRALASIAN TECHNICAL SERVICES NSW PTY LTD,AD212177,NSW,A,6/4/2025,"INGLEBURN, NSW",Asbestos removalist,All friable asbestos removal work All non-fri...
7,AZTECH SERVICES AUSTRALIA PTY LTD,2313682,QLD,A,,"YATALA, QLD",Asbestos removalist,The licence holder must ensure that an asbesto...
8,BC LIM INVESTMENTS PTY LTD,H18/02873,VIC,B,,"TULLAMARINE, VIC",Asbestos removalist,Limited to the removal of Non-friable asbestos...
9,BEASY PTY LTD,AD211426,NSW,A,5/12/2023,"YENNORA, NSW",Asbestos removalist,Nil


In [257]:
df_abs_rem.to_csv('ACT_Abs_Removalist.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Asbestos Assessors

In [258]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=asbestos-assessors')
time.sleep(1)

In [259]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

11


In [266]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[5]/lightning-button/button')
#print(xpaths) 

In [262]:
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::5]
    all_names

['ABDULLAH, MUHAMMAD',
 'ADDISON, PAUL GREGORY',
 'ANGELUCCI, Ellyssa',
 'BAMFORD, SCOTT FRANCIS',
 'BARNES, JARED',
 'BEATON, Caitlin Emily',
 'BELL, ROSS LEWIS',
 'BERKIN, Chantelle Louise',
 'BOBINSKAS, JEREMY',
 'BOSOTAS, Christopher Roberts']

In [267]:
licensee_detail=[]

for i in range(1, numbers[1]+1):
    

    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::5]
    #all_names
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(0,all_names[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr[5]/td[5]/lightning-button/button"}
  (Session info: chrome=96.0.4664.55)


In [268]:
licensee_detail[0]

['ABDULLAH, MUHAMMAD',
 'AA00037',
 '',
 'ACT',
 'JANNALI, NSW',
 'Asbestos assessor',
 'Nil']

In [269]:
licensee_detail[-1]

['XUE, Youja',
 'LAA001435',
 '',
 'NSW',
 'EPPING, NSW',
 'Asbestos assessor',
 'Valid only for air monitoring, clearance inspections and issuing clearance certificates associated with asbestos removal work.']

In [270]:
len(licensee_detail)

104

In [271]:
df_abs_ass = pd.DataFrame(licensee_detail, columns = ['Name', 'License number','Expiry Date','State of issue','Business Location','Occupation','Class Condition / Endorsement'])
df_abs_ass

,Name,License number,Expiry Date,State of issue,Business Location,Occupation,Class Condition / Endorsement
0,"ABDULLAH, MUHAMMAD",AA00037,,ACT,"JANNALI, NSW",Asbestos assessor,Nil
1,"ADDISON, PAUL GREGORY",LAA001298,,NSW,"BOX HILL, VIC",Asbestos assessor,"Valid only for air monitoring, clearance inspe..."
2,"ANGELUCCI, Ellyssa",LAA001441,,NSW,"LAKE HEIGHTS, NSW",Asbestos assessor,NIL
3,"BAMFORD, SCOTT FRANCIS",LAA001047,6/5/2024,NSW,"GIRALANG, ACT",Asbestos assessor,NIL
4,"BARNES, JARED",LAA001026,3/4/2024,NSW,"DICKSON, ACT",Asbestos assessor,Nil
5,"BEATON, Caitlin Emily",LAA001496,,NSW,"COOMBS, ACT",Asbestos assessor,Valid only for air monitoring during Class A a...
6,"BELL, ROSS LEWIS",LAA001255,2/2/2022,NSW,"FYSHWICK, ACT",Asbestos assessor,"Valid only for air monitoring, clearance inspe..."
7,"BERKIN, Chantelle Louise",2317070,8/12/2024,QLD,"COOMA, NSW",Asbestos assessor,"Valid only for air monitoring, clearance inspe..."
8,"BOBINSKAS, JEREMY",AA00023,2/3/2021,ACT,"CANBERRA, ACT",Asbestos assessor,Nil
9,"BOSOTAS, Christopher Roberts",AA00043,5/7/2024,ACT,"JERRABOMBERRA, NSW",Asbestos assessor,NIL


In [272]:
df_abs_ass.shape

(104, 7)

In [273]:
df_abs_ass.to_csv('ACT_Abs_Assessors.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Architects

In [274]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=architects')
time.sleep(1)

In [275]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

35


In [276]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[4]/lightning-button/button')
                     
#print(xpaths) 

In [278]:
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::4]
    all_names

['ABBOTT, OWEN',
 'ABDUL LATIFF, MUHAMMAD AZLI',
 'ADAMSON, PETER',
 'ADHAMI, NABIL',
 'AIFANTIS, NICK',
 'ANASTASIADIS, NICHOLAS',
 'ANDERSON, RAITH STEPHEN',
 'ANDREWS, JAMES ALLAN',
 'APPLEGATE, KAVAN',
 'AUSTIN, MARK ROBERT']

In [279]:
licensee_detail=[]

for i in range(1, numbers[1]+1):
    

    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::4]
    #all_names
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(0,all_names[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr[8]/td[4]/lightning-button/button"}
  (Session info: chrome=96.0.4664.55)


In [280]:
licensee_detail[0]

['ABBOTT, OWEN', '2647', '10/6/2022', 'Architect', 'YARRALUMLA ACT 2600']

In [281]:
licensee_detail[-1]

['ZUBER, TIMOTHY ANDREW', '2384', '31/1/2022', 'Architect', 'PHILIP ACT 2606']

In [282]:
len(licensee_detail)

347

In [283]:
df_architect = pd.DataFrame(licensee_detail, columns = ['Name', 'License number','Expiry Date','Occupation','Business Location'])
df_architect

,Name,License number,Expiry Date,Occupation,Business Location
0,"ABBOTT, OWEN",2647,10/6/2022,Architect,YARRALUMLA ACT 2600
1,"ABDUL LATIFF, MUHAMMAD AZLI",2703,27/10/2022,Architect,MONCRIEFF ACT 2914
2,"ADAMSON, PETER",2612,26/8/2022,Architect,TURNER ACT 2612
3,"ADHAMI, NABIL",2308,13/8/2022,Architect,CANBERRA CITY ACT 2601
4,"AIFANTIS, NICK",2227,13/12/2021,Architect,BARTON ACT 2600
5,"ANASTASIADIS, NICHOLAS",857,5/3/2022,Architect,
6,"ANDERSON, RAITH STEPHEN",2476,19/1/2022,Architect,SURFERS PARADISE QLD 4217
7,"ANDREWS, JAMES ALLAN",1065,24/1/2022,Architect,KINGSTON ACT 2604
8,"APPLEGATE, KAVAN",2631,23/2/2022,Architect,CAMBERWELL VIC 3124
9,"AUSTIN, MARK ROBERT",2577,23/8/2022,Architect,FITZROY VIC 3065


In [284]:
df_architect.shape

(347, 5)

In [285]:
df_architect.to_csv('ACT_Architects.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Certifiers

In [310]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=certifiers')
time.sleep(1)

In [312]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

12


In [313]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[5]/lightning-button/button')
                     
#print(xpaths) 

In [314]:
licensee_detail=[]

for i in range(1, numbers[1]+1):
    

    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::5]
    #all_names
    
    data=data[3:]
    #data
    all_class_desc= []
    all_class_desc= data[::5]
    #all_class_desc
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(0,all_names[k-1])
        ab.insert(2,all_class_desc[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button"}
  (Session info: chrome=96.0.4664.55)


In [315]:
licensee_detail[0]

['ABSOLUTE APPROVALS PTY LTD',
 '2019816',
 'Principal Building Surveyor',
 '30/8/2022',
 'Building Surveyor',
 '']

In [316]:
licensee_detail[-1]

['WRIGLEY, SCOTT ANDREW CHARLES',
 '2016756',
 'General Building Surveyor',
 '18/6/2022',
 'Building Surveyor',
 '']

In [317]:
len(licensee_detail)

120

In [325]:
df_certifier = pd.DataFrame(licensee_detail, columns = ['Name', 'License number','Class Description','Expiry Date','Occupation','Class Condition1','Class Condition2'])
df_certifier

,Name,License number,Class Description,Expiry Date,Occupation,Class Condition1,Class Condition2
0,ABSOLUTE APPROVALS PTY LTD,2019816,Principal Building Surveyor,30/8/2022,Building Surveyor,,None
1,ACT CERTIFICATION SERVICES PTY LTD,2018757,General Building Surveyor,19/6/2022,Building Surveyor,,None
2,ACT METROPOLITAN BUILDING CERTIFIERS PTY LTD,200428123,Principal Building Surveyor,18/11/2021,Building Surveyor,,None
3,"ALLAN, JAMES GORDON",199715300,Plumbing Plan Certifier,14/2/2022,Plumbing Plan Certifier,,None
4,"AMEY, BRETT RICHARD",199915913,Plumbing Plan Certifier,30/9/2022,Plumbing Plan Certifier,'Only valid as an employee of Waterland PTY LTD.',None
5,"ANDERSON, DEAN JOHN",199214659,Plumbing Plan Certifier,15/7/2022,Plumbing Plan Certifier,,None
6,"ANLEZARK, IAN",199816240,Principal Building Surveyor,30/12/2021,Building Surveyor,,None
7,"APOSTOLOPOULOS, CON",2008953,Principal Building Surveyor Employee,30/6/2022,Building Surveyor,For work as an employee of PHILIP CHUN BC VIC ...,None
8,AUSCERT BUILDING CERTIFIERS,2017963,General Building Surveyor,27/9/2022,Building Surveyor,,None
9,"BAILEY, ERIC JOHN",2005553,Principal Building Surveyor,27/6/2022,Building Surveyor,,None


In [326]:
df_certifier['Class Condition1'].unique()

array(['', "'Only valid as an employee of Waterland PTY LTD.'",
       'For work as an employee of PHILIP CHUN BC VIC PTY LTD only.',
       'Nil',
       'Only valid for work as an employee of Certified Building Solutions Pty Ltd, CBS Residential Certifiers Pty Ltd and CBS Commercial Certifiers Pty Ltd.',
       'Valid for work performed on behalf of McKenzie Consulting Group (NSW) Pty Ltd',
       'Restricted to work performed as an employee of JAZ BUILDING CONSULTANTS PTY LTD.',
       'For work as an employee of Du Chateau Chun Pty Ltd only.',
       'Restricted to work performed as an employee of Atelier Consultancy',
       'Valid for work performed as an employee/Nominee only. Not able to contract',
       'Only valid for work as an employee of BCA Certifiers Pty Ltd.',
       'Restricted to work performed as an employee of Hendry Group Pty Ltd only.',
       'Only valid for work performed on behalf of BCA Certifiers Australia Pty Ltd (200714).',
       'Restricted to perform wo

In [327]:
df_certifier.drop(['Class Condition2'], axis = 1)

,Name,License number,Class Description,Expiry Date,Occupation,Class Condition1
0,ABSOLUTE APPROVALS PTY LTD,2019816,Principal Building Surveyor,30/8/2022,Building Surveyor,
1,ACT CERTIFICATION SERVICES PTY LTD,2018757,General Building Surveyor,19/6/2022,Building Surveyor,
2,ACT METROPOLITAN BUILDING CERTIFIERS PTY LTD,200428123,Principal Building Surveyor,18/11/2021,Building Surveyor,
3,"ALLAN, JAMES GORDON",199715300,Plumbing Plan Certifier,14/2/2022,Plumbing Plan Certifier,
4,"AMEY, BRETT RICHARD",199915913,Plumbing Plan Certifier,30/9/2022,Plumbing Plan Certifier,'Only valid as an employee of Waterland PTY LTD.'
5,"ANDERSON, DEAN JOHN",199214659,Plumbing Plan Certifier,15/7/2022,Plumbing Plan Certifier,
6,"ANLEZARK, IAN",199816240,Principal Building Surveyor,30/12/2021,Building Surveyor,
7,"APOSTOLOPOULOS, CON",2008953,Principal Building Surveyor Employee,30/6/2022,Building Surveyor,For work as an employee of PHILIP CHUN BC VIC ...
8,AUSCERT BUILDING CERTIFIERS,2017963,General Building Surveyor,27/9/2022,Building Surveyor,
9,"BAILEY, ERIC JOHN",2005553,Principal Building Surveyor,27/6/2022,Building Surveyor,


In [328]:
df_certifier.shape

(120, 7)

In [329]:
df_certifier.to_csv('ACT_Certifiers.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Gas appliance workers

In [347]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=gas-appliance-workers')
time.sleep(1)

In [348]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

19


In [349]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[5]/lightning-button/button')
                     
#print(xpaths) 

In [351]:
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::5]
    #all_names
    
    data=data[3:]
    #data
    all_class_desc= []
    all_class_desc= data[::5]
    #all_class_desc

In [352]:
licensee_detail=[]

for i in range(1, numbers[1]+1):
    

    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
    all_names= []
    all_names= data[::5]
    #all_names
    
    data=data[3:]
    #data
    all_class_desc= []
    all_class_desc= data[::5]
    #all_class_desc
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(0,all_names[k-1])
        ab.insert(2,all_class_desc[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr[2]/td[5]/lightning-button/button"}
  (Session info: chrome=96.0.4664.55)


In [353]:
licensee_detail[0]

['ABRAMS, MICHAEL',
 '2015341',
 'Unrestricted Type B',
 'Gas Appliance Worker',
 '22 September 2022',
 'Licensee not authorised to install Type B Appliance Flue.']

In [354]:
licensee_detail[-1]

['ZELLER, LIAM DELANEY',
 '20201479',
 'Restricted Type A',
 'Gas Appliance Worker',
 '28 October 2023',
 '']

In [355]:
len(licensee_detail)

181

In [402]:
df_gas_app_wrkr = pd.DataFrame(licensee_detail, columns = ['Name', 'License number','Class Description','Occupation','Expiry Date','Class Condition / Endorsement','h','j'])
df_gas_app_wrkr

,Name,License number,Class Description,Occupation,Expiry Date,Class Condition / Endorsement,h,j
0,"ABRAMS, MICHAEL",2015341,Unrestricted Type B,Gas Appliance Worker,22 September 2022,Licensee not authorised to install Type B Appl...,None,None
1,"AEBERHARD, PASCAL",2013749,Restricted Type B -Level 2,Gas Appliance Worker,28 June 2024,,None,None
2,"ALLARD, STEVEN RODNEY",2009576,Unrestricted Type B,Gas Appliance Worker,22 April 2024,,,None
3,"ALLARD, STEVEN RODNEY",2009576,Restricted Type A,Gas Appliance Worker,25 November 2022,,,None
4,"ANDERSON, STEVEN",2011299,Restricted Type B - Level 1,Gas Appliance Worker,19 August 2022,,None,None
5,"ASHWORTH, DAVID PATRICK",2015340,Unrestricted Type A,Gas Appliance Worker,2 July 2023,,None,None
6,"BACKHOUSE, JOHN RAYMOND",2015342,Unrestricted Type A,Gas Appliance Worker,11 July 2024,,None,None
7,"BAILEY, MICHAEL TERENCE",2009827,Unrestricted Type A,Gas Appliance Worker,8 September 2023,,,None
8,"BAILEY, MICHAEL TERENCE",2009827,Unrestricted Type B,Gas Appliance Worker,8 September 2023,,,None
9,"BAILEY, IAN WILLIAM",199613974,Unrestricted Type A,Gas Appliance Worker,20 January 2023,,None,None


In [403]:
df_gas_app_wrkr["Class Condition / Endorsement"] = df_gas_app_wrkr["Class Condition / Endorsement"] + ',' + df_gas_app_wrkr["h"].astype(str) + ',' + df_gas_app_wrkr["j"].astype(str)
df_gas_app_wrkr

,Name,License number,Class Description,Occupation,Expiry Date,Class Condition / Endorsement,h,j
0,"ABRAMS, MICHAEL",2015341,Unrestricted Type B,Gas Appliance Worker,22 September 2022,Licensee not authorised to install Type B Appl...,None,None
1,"AEBERHARD, PASCAL",2013749,Restricted Type B -Level 2,Gas Appliance Worker,28 June 2024,",None,None",None,None
2,"ALLARD, STEVEN RODNEY",2009576,Unrestricted Type B,Gas Appliance Worker,22 April 2024,",,None",,None
3,"ALLARD, STEVEN RODNEY",2009576,Restricted Type A,Gas Appliance Worker,25 November 2022,",,None",,None
4,"ANDERSON, STEVEN",2011299,Restricted Type B - Level 1,Gas Appliance Worker,19 August 2022,",None,None",None,None
5,"ASHWORTH, DAVID PATRICK",2015340,Unrestricted Type A,Gas Appliance Worker,2 July 2023,",None,None",None,None
6,"BACKHOUSE, JOHN RAYMOND",2015342,Unrestricted Type A,Gas Appliance Worker,11 July 2024,",None,None",None,None
7,"BAILEY, MICHAEL TERENCE",2009827,Unrestricted Type A,Gas Appliance Worker,8 September 2023,",,None",,None
8,"BAILEY, MICHAEL TERENCE",2009827,Unrestricted Type B,Gas Appliance Worker,8 September 2023,",,None",,None
9,"BAILEY, IAN WILLIAM",199613974,Unrestricted Type A,Gas Appliance Worker,20 January 2023,",None,None",None,None


In [404]:
df_gas_app_wrkr["Class Condition / Endorsement"][18]

',None,None'

In [405]:
df_gas_app_wrkr=df_gas_app_wrkr.drop(['h','j'], axis = 1)
df_gas_app_wrkr

,Name,License number,Class Description,Occupation,Expiry Date,Class Condition / Endorsement
0,"ABRAMS, MICHAEL",2015341,Unrestricted Type B,Gas Appliance Worker,22 September 2022,Licensee not authorised to install Type B Appl...
1,"AEBERHARD, PASCAL",2013749,Restricted Type B -Level 2,Gas Appliance Worker,28 June 2024,",None,None"
2,"ALLARD, STEVEN RODNEY",2009576,Unrestricted Type B,Gas Appliance Worker,22 April 2024,",,None"
3,"ALLARD, STEVEN RODNEY",2009576,Restricted Type A,Gas Appliance Worker,25 November 2022,",,None"
4,"ANDERSON, STEVEN",2011299,Restricted Type B - Level 1,Gas Appliance Worker,19 August 2022,",None,None"
5,"ASHWORTH, DAVID PATRICK",2015340,Unrestricted Type A,Gas Appliance Worker,2 July 2023,",None,None"
6,"BACKHOUSE, JOHN RAYMOND",2015342,Unrestricted Type A,Gas Appliance Worker,11 July 2024,",None,None"
7,"BAILEY, MICHAEL TERENCE",2009827,Unrestricted Type A,Gas Appliance Worker,8 September 2023,",,None"
8,"BAILEY, MICHAEL TERENCE",2009827,Unrestricted Type B,Gas Appliance Worker,8 September 2023,",,None"
9,"BAILEY, IAN WILLIAM",199613974,Unrestricted Type A,Gas Appliance Worker,20 January 2023,",None,None"


In [406]:
df_gas_app_wrkr["Class Condition / Endorsement"][0]

'Licensee not authorised to install Type B Appliance Flue.,None,None'

In [407]:
df_gas_app_wrkr.shape

(181, 6)

In [408]:
df_gas_app_wrkr.to_csv('ACT_Gas_app_workers.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Building assessors

In [428]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=building-assessors-public-register')
time.sleep(1)

In [429]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

7


In [430]:
names=[]
lic_no=[]
exp=[]
occ=[]
cl_desc=[]
condtn=[]

for i in range(1, numbers[1]+1):
 
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
  
    all_names= []
    all_names= data[::7]
    #all_names
    names=names + all_names
    
    data=data[1:]
    #data
    all_lic_no= []
    all_lic_no= data[::7]
    #all_lic_no
    lic_no = lic_no + all_lic_no
    
    data=data[1:]
    #data
    all_exp= []
    all_exp= data[::7]
    #all_exp
    exp = exp + all_exp
    
    data=data[1:]
    #data
    all_occ= []
    all_occ= data[::7]
    #all_occ
    occ = occ + all_occ
    
    data=data[1:]
    #data
    all_cl_desc= []
    all_cl_desc= data[::7]
    #all_cl_desc
    cl_desc = cl_desc + all_cl_desc
    
    data=data[1:]
    #data
    all_condtn= []
    all_condtn= data[::7]
    #all_condtn  
    condtn = condtn + all_condtn
    
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button"}
  (Session info: chrome=96.0.4664.55)


In [431]:
df_build_ass=pd.DataFrame(list(zip(names,lic_no,exp,occ,cl_desc,condtn)), columns=['Name','Licence No','Expiry Date','Occupation','Class Description','Class Condition / Endorsement'])
df_build_ass

,Name,Licence No,Expiry Date,Occupation,Class Description,Class Condition / Endorsement
0,"ADAMSON, PETER JOHN",19905019,6 April 2022,Building Assessor,Class A - Energy Efficiency,Operation of BERS Professional version 4.1 and...
1,"AKBARI, SULAIMAN",2011217,17 March 2022,Building Assessor,Class B - Energy Efficiency,
2,"BEAUCHESNE, CORY",2017793,2 October 2022,Building Assessor,Class B - Energy Efficiency,Operation of FirstRate 5 software version 5.1
3,"BHUDIA, DIXIT BHIMJI",20211456,17 September 2022,Building Assessor,Class A - Energy Efficiency,Operation of First Rate 5 version 5.3.0(3.21)
4,"BURR, ANTHONY DAVID",199514030,6 March 2022,Building Assessor,Class A - Energy Efficiency,Operation of BERS Professional Version 4.1 and...
5,"BYATT, PETER RONALD",1973901,24 February 2022,Building Assessor,Class A - Energy Efficiency,Please contact Access Canberra for condition d...
6,"CAMPBELL, TRISHA LEE",2011186,31 March 2022,Building Assessor,Class A - Energy Efficiency,Operation of BERS Professional Version 4.2 and...
7,"CAMPBELL, STUART",2019167,28 May 2022,Building Assessor,Class A - Energy Efficiency,Endorsed for the operation of FirstRate 5 soft...
8,"CECH, MIROSLAV",20201003,22 September 2022,Building Assessor,Class A - Energy Efficiency,Endorsed for the operation of FirstRate 5 soft...
9,"CHAMPNESS, ANDREW DAVID",20121191,30 April 2022,Building Assessor,Class A - Energy Efficiency,Operation of FirstRate 5


In [433]:
df_build_ass.shape

(64, 6)

In [434]:
df_build_ass.to_csv('ACT_Building_Assessors.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**

## ACT Registered land surveyors

In [435]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=registered-land-surveyors')
time.sleep(1)

In [436]:
pages=driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages
numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1])

10


In [446]:
names=[]
lic_no=[]
occ=[]

for i in range(1, numbers[1]+1):
 
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    data = [j.text for j in soup.findAll('td')]
    #data
  
    all_names= []
    all_names= data[::4]
    #all_names
    names = names + all_names
    
    data=data[1:]
    #data
    all_lic_no= []
    all_lic_no= data[::4]
    #all_lic_no
    lic_no = lic_no + all_lic_no
    
    data=data[1:]
    #data
    all_occ= []
    all_occ= data[::4]
    #all_occ
    occ = occ + all_occ
    
    driver.find_element_by_xpath('/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button"}
  (Session info: chrome=96.0.4664.55)


In [447]:
df_land_sur=pd.DataFrame(list(zip(names,lic_no,occ)), columns=['Name','Licence No','Occupation'])
df_land_sur

,Name,Licence No,Occupation
0,"Allison, Scott",403,Land surveyor
1,"Allpress, Philip",344,Land surveyor
2,"Anastovski, Toni",443,Land surveyor
3,"Atkinson, Jack",459,Land surveyor
4,"Bardsley, Alan",429,Land surveyor
5,"Beljanski, Steven",294,Land surveyor
6,"Bonisch, James",448,Land surveyor
7,"Bradley, Adam",422,Land surveyor
8,"Brown, Rachel",373,Land surveyor
9,"Brown, Jeffrey",276,Land surveyor


In [448]:
df_land_sur.shape

(93, 3)

In [449]:
df_land_sur.to_csv('ACT_Land_surveyors.csv')

# **XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX**